# Regression Week 4: Ridge Regression (interpretation)

In this notebook, we will run ridge regression multiple times with different L2 penalties to see which one produces the best fit. We will revisit the example of polynomial regression as a means to see the effect of L2 regularization. In particular, we will:
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression
* Use matplotlib to visualize polynomial regressions
* Use a pre-built implementation of regression (GraphLab Create) to run polynomial regression, this time with L2 penalty
* Use matplotlib to visualize polynomial regressions under L2 regularization
* Choose best L2 penalty using cross-validation.
* Assess the final fit using test data.

We will continue to use the House data from previous notebooks.  (In the next programming assignment for this module, you will implement your own ridge regression learning algorithm using gradient descent.)

# Fire up graphlab create

In [1]:
import graphlab
import numpy

# Polynomial regression, revisited

We build on the material from Week 3, where we wrote the function to produce an SFrame with columns containing the powers of a given input. Copy and paste the function `polynomial_sframe` from Week 3:

In [2]:
def polynomial_sframe(feature, degree):
    # assume that degree >= 1
    # initialize the SFrame:
    poly_sframe = graphlab.SFrame()
    # and set poly_sframe['power_1'] equal to the passed feature
    poly_sframe['power_1'] = feature
    # first check if degree > 1
    if degree > 1:
        # then loop over the remaining degrees:
        # range usually starts at 0 and stops at the endpoint-1. We want it to start at 2 and stop at degree
        for power in range(2, degree+1): 
            # first we'll give the column a name:
            name = 'power_' + str(power)
            # then assign poly_sframe[name] to the appropriate power of feature
            poly_sframe[name] = poly_sframe.apply(lambda row: pow(row['power_1'], power) )
    return poly_sframe    

Let's use matplotlib to visualize what a polynomial regression looks like on the house data.

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1483717145.log


This non-commercial license of GraphLab Create for academic use is assigned to julian.trajanson@gmail.com and will expire on September 29, 2017.


In [4]:
sales = graphlab.SFrame('kc_house_data.gl/')

As in Week 3, we will use the sqft_living variable. For plotting purposes (connecting the dots), you'll need to sort by the values of sqft_living. For houses with identical square footage, we break the tie by their prices.

In [5]:
sales = sales.sort(['sqft_living','price'])

Let us revisit the 15th-order polynomial model using the 'sqft_living' input. Generate polynomial features up to degree 15 using `polynomial_sframe()` and fit a model with these features. When fitting the model, use an L2 penalty of `1e-5`:

In [6]:
l2_small_penalty = 1e-5

Note: When we have so many features and so few data points, the solution can become highly numerically unstable, which can sometimes lead to strange unpredictable results.  Thus, rather than using no regularization, we will introduce a tiny amount of regularization (`l2_penalty=1e-5`) to make the solution numerically stable.  (In lecture, we discussed the fact that regularization can also help with numerical stability, and here we are seeing a practical example.)

With the L2 penalty specified above, fit the model and print out the learned weights.

Hint: make sure to add 'price' column to the new SFrame before calling `graphlab.linear_regression.create()`. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set=None` in this call.

In [7]:
data = polynomial_sframe(sales['sqft_living'], 15)
data['price'] = sales['price']

print data

+---------+----------+------------+---------------+-------------------+
| power_1 | power_2  |  power_3   |    power_4    |      power_5      |
+---------+----------+------------+---------------+-------------------+
|  290.0  | 84100.0  | 24389000.0 |  7072810000.0 |   2.0511149e+12   |
|  370.0  | 136900.0 | 50653000.0 | 18741610000.0 |   6.9343957e+12   |
|  380.0  | 144400.0 | 54872000.0 | 20851360000.0 |   7.9235168e+12   |
|  384.0  | 147456.0 | 56623104.0 | 21743271936.0 | 8.34941642342e+12 |
|  390.0  | 152100.0 | 59319000.0 | 23134410000.0 |   9.0224199e+12   |
|  390.0  | 152100.0 | 59319000.0 | 23134410000.0 |   9.0224199e+12   |
|  410.0  | 168100.0 | 68921000.0 | 28257610000.0 |   1.15856201e+13  |
|  420.0  | 176400.0 | 74088000.0 | 31116960000.0 |   1.30691232e+13  |
|  420.0  | 176400.0 | 74088000.0 | 31116960000.0 |   1.30691232e+13  |
|  430.0  | 184900.0 | 79507000.0 | 34188010000.0 |   1.47008443e+13  |
+---------+----------+------------+---------------+-------------

In [8]:
def polynomial_ridge_regression(data, deg, l2_penalty):
    model = graphlab.linear_regression.create(data, 
                                              target='price', l2_penalty=l2_penalty,
                                              validation_set=None,verbose=False)
    return model

def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients['value'])-1

    # Get learned parameters as a list
    w = list(model.coefficients['value'])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print 'Learned polynomial for degree ' + str(deg) + ':'
    w.reverse()
    print numpy.poly1d(w)

In [11]:
model = polynomial_ridge_regression(data, 15, 1e-5)
print model
# print_coefficients(model)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 21613
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0282

Settings
--------
Residual sum of squares        : 1.30428181617e+15
Training RMSE                  : 245656.4622

Highest Positive Coefficients
-----------------------------
(intercept)                    : 167924.8659
power_1                        : 103.0909
power_2                        : 0.1346
power_4                        : 0.0
power_6                        : 0.0

Lowest Negative Coefficients
----------------------------
power_3                      

***QUIZ QUESTION:  What's the learned value for the coefficient of feature `power_1`?***

# Observe overfitting

Recall from Week 3 that the polynomial fit of degree 15 changed wildly whenever the data changed. In particular, when we split the sales data into four subsets and fit the model of degree 15, the result came out to be very different for each subset. The model had a *high variance*. We will see in a moment that ridge regression reduces such variance. But first, we must reproduce the experiment we did in Week 3.

First, split the data into split the sales data into four subsets of roughly equal size and call them `set_1`, `set_2`, `set_3`, and `set_4`. Use `.random_split` function and make sure you set `seed=0`. 

In [15]:
(semi_split1, semi_split2) = sales.random_split(.5,seed=0)
(set_1, set_2) = semi_split1.random_split(0.5, seed=0)
(set_3, set_4) = semi_split2.random_split(0.5, seed=0)

Next, fit a 15th degree polynomial on `set_1`, `set_2`, `set_3`, and `set_4`, using 'sqft_living' to predict prices. Print the weights and make a plot of the resulting model.

Hint: When calling `graphlab.linear_regression.create()`, use the same L2 penalty as before (i.e. `l2_small_penalty`).  Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [18]:
def generate_model(input_data, degree, l2_penalty):
    data = polynomial_sframe(input_data['sqft_living'], degree)
    data['price'] = input_data['price']
    model = polynomial_ridge_regression(data, degree, l2_penalty)
    print model

In [19]:
generate_model(set_1, 15, 1e-5)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5404
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0459

Settings
--------
Residual sum of squares        : 3.34244159985e+14
Training RMSE                  : 248699.1173

Highest Positive Coefficients
-----------------------------
(intercept)                    : 9306.4655
power_1                        : 585.8658
power_3                        : 0.0001
power_6                        : 0.0
power_7                        : 0.0

Lowest Negative Coefficients
----------------------------
power_2                        :

In [21]:
generate_model(set_2, 15, 1e-5)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5398
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0697

Settings
--------
Residual sum of squares        : 2.96922466393e+14
Training RMSE                  : 234533.6106

Highest Positive Coefficients
-----------------------------
power_1                        : 783.4938
power_3                        : 0.0004
power_5                        : 0.0
power_6                        : 0.0
power_10                       : 0.0

Lowest Negative Coefficients
----------------------------
(intercept)                    : -2511

In [22]:
generate_model(set_3, 15, 1e-5)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5409
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0427

Settings
--------
Residual sum of squares        : 3.41037823442e+14
Training RMSE                  : 251097.7281

Highest Positive Coefficients
-----------------------------
(intercept)                    : 462426.5678
power_2                        : 1.0287
power_4                        : 0.0
power_7                        : 0.0
power_8                        : 0.0

Lowest Negative Coefficients
----------------------------
power_1                        : -7

In [20]:
generate_model(set_4, 15, 1e-5)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5402
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0571

Settings
--------
Residual sum of squares        : 3.22513810178e+14
Training RMSE                  : 244341.2932

Highest Positive Coefficients
-----------------------------
power_1                        : 1247.5903
power_3                        : 0.0006
power_6                        : 0.0
power_7                        : 0.0
power_10                       : 0.0

Lowest Negative Coefficients
----------------------------
(intercept)                    : -170

The four curves should differ from one another a lot, as should the coefficients you learned.

***QUIZ QUESTION:  For the models learned in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?***  (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Ridge regression comes to rescue

Generally, whenever we see weights change so much in response to change in data, we believe the variance of our estimate to be large. Ridge regression aims to address this issue by penalizing "large" weights. (Weights of `model15` looked quite small, but they are not that small because 'sqft_living' input is in the order of thousands.)

With the argument `l2_penalty=1e5`, fit a 15th-order polynomial model on `set_1`, `set_2`, `set_3`, and `set_4`. Other than the change in the `l2_penalty` parameter, the code should be the same as the experiment above. Also, make sure GraphLab Create doesn't create its own validation set by using the option `validation_set = None` in this call.

In [23]:
generate_model(set_1, 15, 1e5)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5404
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 100000.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0476

Settings
--------
Residual sum of squares        : 7.5694820013e+14
Training RMSE                  : 374261.7209

Highest Positive Coefficients
-----------------------------
(intercept)                    : 530317.0245
power_1                        : 2.5874
power_2                        : 0.0013
power_3                        : 0.0
power_4                        : 0.0

Lowest Negative Coefficients
----------------------------
No Negative Coefficients    

In [24]:
generate_model(set_2, 15, 1e5)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5398
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 100000.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0636

Settings
--------
Residual sum of squares        : 5.64001004832e+14
Training RMSE                  : 323238.8096

Highest Positive Coefficients
-----------------------------
(intercept)                    : 519216.8974
power_1                        : 2.0447
power_2                        : 0.0011
power_3                        : 0.0
power_4                        : 0.0

Lowest Negative Coefficients
----------------------------
No Negative Coefficients   

In [25]:
generate_model(set_3, 15, 1e5)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5409
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 100000.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0416

Settings
--------
Residual sum of squares        : 6.62729427752e+14
Training RMSE                  : 350033.5213

Highest Positive Coefficients
-----------------------------
(intercept)                    : 522911.518
power_1                        : 2.2689
power_2                        : 0.0013
power_3                        : 0.0
power_4                        : 0.0

Lowest Negative Coefficients
----------------------------
No Negative Coefficients    

In [26]:
generate_model(set_4, 15, 1e5)

Class                          : LinearRegression

Schema
------
Number of coefficients         : 16
Number of examples             : 5402
Number of feature columns      : 15
Number of unpacked features    : 15

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 100000.0

Training Summary
----------------
Solver                         : newton
Solver iterations              : 1
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0598

Settings
--------
Residual sum of squares        : 5.63974715174e+14
Training RMSE                  : 323111.5829

Highest Positive Coefficients
-----------------------------
(intercept)                    : 513667.0871
power_1                        : 1.9104
power_2                        : 0.0011
power_3                        : 0.0
power_4                        : 0.0

Lowest Negative Coefficients
----------------------------
No Negative Coefficients   

These curves should vary a lot less, now that you applied a high degree of regularization.

***QUIZ QUESTION:  For the models learned with the high level of regularization in each of these training sets, what are the smallest and largest values you learned for the coefficient of feature `power_1`?*** (For the purpose of answering this question, negative numbers are considered "smaller" than positive numbers. So -5 is smaller than -3, and -3 is smaller than 5 and so forth.)

# Selecting an L2 penalty via cross-validation

Just like the polynomial degree, the L2 penalty is a "magic" parameter we need to select. We could use the validation set approach as we did in the last module, but that approach has a major disadvantage: it leaves fewer observations available for training. **Cross-validation** seeks to overcome this issue by using all of the training set in a smart way.

We will implement a kind of cross-validation called **k-fold cross-validation**. The method gets its name because it involves dividing the training set into k segments of roughtly equal size. Similar to the validation set method, we measure the validation error with one of the segments designated as the validation set. The major difference is that we repeat the process k times as follows:

Set aside segment 0 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
Set aside segment 1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set<br>
...<br>
Set aside segment k-1 as the validation set, and fit a model on rest of data, and evalutate it on this validation set

After this process, we compute the average of the k validation errors, and use it as an estimate of the generalization error. Notice that  all observations are used for both training and validation, as we iterate over segments of data. 

To estimate the generalization error well, it is crucial to shuffle the training data before dividing them into segments. GraphLab Create has a utility function for shuffling a given SFrame. We reserve 10% of the data as the test set and shuffle the remainder. (Make sure to use `seed=1` to get consistent answer.)

In [27]:
(train_valid, test) = sales.random_split(.9, seed=1)
train_valid_shuffled = graphlab.toolkits.cross_validation.shuffle(train_valid, random_seed=1)

Once the data is shuffled, we divide it into equal segments. Each segment should receive `n/k` elements, where `n` is the number of observations in the training set and `k` is the number of segments. Since the segment 0 starts at index 0 and contains `n/k` elements, it ends at index `(n/k)-1`. The segment 1 starts where the segment 0 left off, at index `(n/k)`. With `n/k` elements, the segment 1 ends at index `(n*2/k)-1`. Continuing in this fashion, we deduce that the segment `i` starts at index `(n*i/k)` and ends at `(n*(i+1)/k)-1`.

With this pattern in mind, we write a short loop that prints the starting and ending indices of each segment, just to make sure you are getting the splits right.

In [86]:
n = len(train_valid_shuffled)
k = 10 # 10-fold cross-validation

k_10_ranges = []

for i in xrange(k):
    start = (n*i)/k
    end = (n*(i+1))/k-1
    k_10_ranges.append( (start, end) )
    print i, (start, end)

0 (0, 1938)
1 (1939, 3878)
2 (3879, 5817)
3 (5818, 7757)
4 (7758, 9697)
5 (9698, 11636)
6 (11637, 13576)
7 (13577, 15515)
8 (15516, 17455)
9 (17456, 19395)


Let us familiarize ourselves with array slicing with SFrame. To extract a continuous slice from an SFrame, use colon in square brackets. For instance, the following cell extracts rows 0 to 9 of `train_valid_shuffled`. Notice that the first index (0) is included in the slice but the last index (10) is omitted.

In [87]:
train_valid_shuffled[0:10] # rows 0 to 9

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
2780400035,2014-05-05 00:00:00+00:00,665000.0,4.0,2.5,2800.0,5900,1,0
1703050500,2015-03-21 00:00:00+00:00,645000.0,3.0,2.5,2490.0,5978,2,0
5700002325,2014-06-05 00:00:00+00:00,640000.0,3.0,1.75,2340.0,4206,1,0
0475000510,2014-11-18 00:00:00+00:00,594000.0,3.0,1.0,1320.0,5000,1,0
0844001052,2015-01-28 00:00:00+00:00,365000.0,4.0,2.5,1904.0,8200,2,0
2658000373,2015-01-22 00:00:00+00:00,305000.0,4.0,2.0,1610.0,6250,1,0
3750603471,2015-03-27 00:00:00+00:00,239950.0,3.0,2.5,1560.0,4800,2,0
2114700540,2014-10-21 00:00:00+00:00,366000.0,3.0,2.5,1320.0,4320,1,0
2596400050,2014-07-30 00:00:00+00:00,375000.0,3.0,1.0,1960.0,7955,1,0
4140900050,2015-01-26 00:00:00+00:00,440000.0,4.0,1.75,2180.0,10200,1,0


Now let us extract individual segments with array slicing. Consider the scenario where we group the houses in the `train_valid_shuffled` dataframe into k=10 segments of roughly equal size, with starting and ending indices computed as above.
Extract the fourth segment (segment 3) and assign it to a variable called `validation4`.

In [88]:
def get_data_segement(index, ranges):
    return train_valid_shuffled[ranges[index][0]:ranges[index][1]+1]
    
validation4 = get_data_segement(3, k_10_ranges)

print validation4

+------------+---------------------------+----------+----------+-----------+
|     id     |            date           |  price   | bedrooms | bathrooms |
+------------+---------------------------+----------+----------+-----------+
| 3342102880 | 2014-08-11 00:00:00+00:00 | 464000.0 |   3.0    |    2.5    |
| 5113400364 | 2015-01-26 00:00:00+00:00 | 650000.0 |   4.0    |    1.5    |
| 0984210170 | 2015-03-26 00:00:00+00:00 | 256500.0 |   5.0    |    2.5    |
| 6117501755 | 2014-12-30 00:00:00+00:00 | 355000.0 |   4.0    |    1.5    |
| 4307300520 | 2015-04-23 00:00:00+00:00 | 359000.0 |   4.0    |    2.5    |
| 7616200050 | 2015-01-05 00:00:00+00:00 | 500000.0 |   3.0    |    1.75   |
| 2518400046 | 2014-11-18 00:00:00+00:00 | 456700.0 |   3.0    |    1.75   |
| 2877100235 | 2014-12-22 00:00:00+00:00 | 650000.0 |   5.0    |    2.0    |
| 6400700389 | 2014-07-10 00:00:00+00:00 | 875000.0 |   5.0    |    3.0    |
| 1422200090 | 2014-09-15 00:00:00+00:00 | 676500.0 |   3.0    |    1.75   |

To verify that we have the right elements extracted, run the following cell, which computes the average price of the fourth segment. When rounded to nearest whole number, the average should be $536,234.

In [89]:
print int(round(validation4['price'].mean(), 0))

536234


After designating one of the k segments as the validation set, we train a model using the rest of the data. To choose the remainder, we slice (0:start) and (end+1:n) of the data and paste them together. SFrame has `append()` method that pastes together two disjoint sets of rows originating from a common dataset. For instance, the following cell pastes together the first and last two rows of the `train_valid_shuffled` dataframe.

In [90]:
n = len(train_valid_shuffled)
first_two = train_valid_shuffled[0:2]
last_two = train_valid_shuffled[n-2:n]
print first_two.append(last_two)

+------------+---------------------------+-----------+----------+-----------+
|     id     |            date           |   price   | bedrooms | bathrooms |
+------------+---------------------------+-----------+----------+-----------+
| 2780400035 | 2014-05-05 00:00:00+00:00 |  665000.0 |   4.0    |    2.5    |
| 1703050500 | 2015-03-21 00:00:00+00:00 |  645000.0 |   3.0    |    2.5    |
| 4139480190 | 2014-09-16 00:00:00+00:00 | 1153000.0 |   3.0    |    3.25   |
| 7237300290 | 2015-03-26 00:00:00+00:00 |  338000.0 |   5.0    |    2.5    |
+------------+---------------------------+-----------+----------+-----------+
+-------------+----------+--------+------------+------+-----------+-------+------------+
| sqft_living | sqft_lot | floors | waterfront | view | condition | grade | sqft_above |
+-------------+----------+--------+------------+------+-----------+-------+------------+
|    2800.0   |   5900   |   1    |     0      |  0   |     3     |   8   |    1660    |
|    2490.0   |   59

Extract the remainder of the data after *excluding* fourth segment (segment 3) and assign the subset to `train4`.

In [91]:
def get_remaining_data_segement(index, ranges):
    start = ranges[index][0]
    end   = ranges[index][1]
    start_of_data = train_valid_shuffled[:start]
    end_of_data   = train_valid_shuffled[end + 1:]
    return start_of_data.append(end_of_data)

train4 = get_remaining_data_segement(3, k_10_ranges)

To verify that we have the right elements extracted, run the following cell, which computes the average price of the data with fourth segment excluded. When rounded to nearest whole number, the average should be $539,450.

In [92]:
print int(round(train4['price'].mean(), 0))

539450


Now we are ready to implement k-fold cross-validation. Write a function that computes k validation errors by designating each of the k segments as the validation set. It accepts as parameters (i) `k`, (ii) `l2_penalty`, (iii) dataframe, (iv) name of output column (e.g. `price`) and (v) list of feature names. The function returns the average validation error using k segments as validation sets.

* For each i in [0, 1, ..., k-1]:
  * Compute starting and ending indices of segment i and call 'start' and 'end'
  * Form validation set by taking a slice (start:end+1) from the data.
  * Form training set by appending slice (end+1:n) to the end of slice (0:start).
  * Train a linear model using training set just formed, with a given l2_penalty
  * Compute validation error using validation set just formed

In [116]:
def get_data_for_model(data, features_list, output_name):
    sframe = graphlab.SFrame()
    for feature in features_list:
        sframe[feature] = data[feature]
    sframe[output_name] = data[output_name]
    return sframe
    
def k_fold_cross_validation(k, l2_penalty, data, output_name, features_list):
    validation_errors = []
    ranges = []    
    for i in xrange(k):
        start = (n*i)/k
        end = (n*(i+1))/k-1
        ranges.append( (start, end) )
    
    for validation_index in xrange(k):
        validation_set = get_data_segement(validation_index, k_10_ranges)
        training_set = get_remaining_data_segement(validation_index, k_10_ranges)
        data = get_data_for_model(training_set, features_list, output_name)
        model = graphlab.linear_regression.create(data, 
                                          target=output_name, l2_penalty=l2_penalty,
                                          validation_set=None,verbose=False)
        y_hats = model.predict(validation_set)
        ys     = validation_set[output_name]
        validation_error = ((ys - y_hats) ** 2).sum()
        validation_errors.append( (validation_error) )
    
    return numpy.mean(  numpy.array(validation_errors) )
    
        


Once we have a function to compute the average validation error for a model, we can write a loop to find the model that minimizes the average validation error. Write a loop that does the following:
* We will again be aiming to fit a 15th-order polynomial model using the `sqft_living` input
* For `l2_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, you can use this Numpy function: `np.logspace(1, 7, num=13)`.)
    * Run 10-fold cross-validation with `l2_penalty`
* Report which L2 penalty produced the lowest average validation error.

Note: since the degree of the polynomial is now fixed to 15, to make things faster, you should generate polynomial features in advance and re-use them throughout the loop. Make sure to use `train_valid_shuffled` when generating polynomial features!

In [117]:
l2_validations = numpy.logspace(1, 7, num=13)

min_validation_error = numpy.inf
best_l2 = None

for l2_validation in l2_validations:
    validation_error = k_fold_cross_validation(k=10, l2_penalty=l2_validation, data=train_valid_shuffled, output_name='price', features_list=["sqft_living"])
    print "l2_validation", l2_validation
    print "validation_error", validation_error    
    if validation_error < min_validation_error:
        min_validation_error = validation_error
        best_l2 = l2_validation
print best_l2

l2_validation 10.0
validation_error 1.34914289289e+14
l2_validation 31.6227766017
validation_error 1.34928006114e+14
l2_validation 100.0
validation_error 1.350609858e+14
l2_validation 316.227766017
validation_error 1.36209350887e+14
l2_validation 1000.0
validation_error 1.43711335861e+14
l2_validation 3162.27766017
validation_error 1.71074501984e+14
l2_validation 10000.0
validation_error 2.14209465151e+14
l2_validation 31622.7766017
validation_error 2.45191331574e+14
l2_validation 100000.0
validation_error 2.58844902676e+14
l2_validation 316227.766017
validation_error 2.63693267041e+14
l2_validation 1000000.0
validation_error 2.65285763234e+14
l2_validation 3162277.66017
validation_error 2.65795505066e+14
l2_validation 10000000.0
validation_error 2.6595732179e+14
10.0


***QUIZ QUESTIONS:  What is the best value for the L2 penalty according to 10-fold validation?***

You may find it useful to plot the k-fold cross-validation errors you have obtained to better understand the behavior of the method.  

In [ ]:
# Plot the l2_penalty values in the x axis and the cross-validation error in the y axis.
# Using plt.xscale('log') will make your plot more intuitive.



Once you found the best value for the L2 penalty using cross-validation, it is important to retrain a final model on all of the training data using this value of `l2_penalty`. This way, your final model will be trained on the entire dataset.

***QUIZ QUESTION: Using the best L2 penalty found above, train a model using all training data. What is the RSS on the TEST data of the model you learn with this L2 penalty? ***